In [12]:
import requests
from bs4 import BeautifulSoup
from user_agent import generate_user_agent


url = "https://www.amazon.com.mx/s"  # URL de búsqueda en Amazon México

# Parámetros para la búsqueda
params = {
    'k': 'freidora'  # Término de búsqueda, en este caso "freidora"
}

# Encabezados personalizados
custom_headers = {
    'user-agent': generate_user_agent(),  # Generar un User-Agent aleatorio
    'accept-language': 'en-GB,en;q=0.9', # mejorar los headers 
}

# Lista para almacenar los enlaces de los productos
product_links_list = []
num_products = 500  # Número de productos a recolectar
current_url = url  # URL inicial para comenzar la búsqueda

while len(product_links_list) < num_products:
    response = requests.get(current_url, params=params, headers=custom_headers)
    
    # Comprobamos que la solicitud fue exitosa
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        product_links = soup.find_all("a", class_="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal")

        for product in product_links:
            if len(product_links_list) >= num_products:
                break

            link = "https://www.amazon.com.mx" + product['href']
            product_links_list.append(link)
        
        next_page = soup.find("a", class_="s-pagination-item s-pagination-next s-pagination-button s-pagination-separator")
        
        if next_page and 'href' in next_page.attrs:
            current_url = "https://www.amazon.com.mx" + next_page['href']
        else:
            print("yap quedo o no hay massss ")
            break
    else:
        print("No se pudo acceder a la página. Status code:", response.status_code)
        break

# Imprimir los enlaces de productos recolectados
print(f"Total de enlaces recolectados: {len(product_links_list)}")
for link in product_links_list:
    print(link)


yap quedo o no hay massss 
Total de enlaces recolectados: 390
https://www.amazon.com.mx/Hamilton-Beach-35034-Freidora/dp/B00M39ML76/ref=sr_1_1?dib=eyJ2IjoiMSJ9.AHRpCASmeBcoCU1sKgIWPCS5J0_YJ9R3ivEWYegI6o5RefUvLVuKzj4JDuwdKLwWDHVVEW62qt_cCuiPqtc1X0Eti0bnyzsR33U88J5sSw7Zx10-xT0VIxNitysXvwxHsOz1Vwp6Ep-QwzLZ6foW0JdQmwQ0tdldYgtLHQ3jVdbgJrYydBDvklxy75rkXzZb_UoWrwwbOCy6irdWg6DsvlDXIpmDnjpH_Mjx8c7t-ESX99zhwnhG8j1rGzq6whgCcxHGvpcIw8I-Jrr5lxm2ByIvPdG2lIupz92ucXUh_VM.JAvtn_4JSlSKzJAgkBLxBtKoXuin23i7kuAX5j5-rYM&dib_tag=se&keywords=freidora&qid=1730093152&sr=8-1&ufe=app_do%3Aamzn1.fos.3ad1a8a7-7417-4467-b505-11834e377abf
https://www.amazon.com.mx/Hamilton-Beach-35210-Profesional-Inoxidable/dp/B08YMWQF2X/ref=sr_1_2?dib=eyJ2IjoiMSJ9.AHRpCASmeBcoCU1sKgIWPCS5J0_YJ9R3ivEWYegI6o5RefUvLVuKzj4JDuwdKLwWDHVVEW62qt_cCuiPqtc1X0Eti0bnyzsR33U88J5sSw7Zx10-xT0VIxNitysXvwxHsOz1Vwp6Ep-QwzLZ6foW0JdQmwQ0tdldYgtLHQ3jVdbgJrYydBDvklxy75rkXzZb_UoWrwwbOCy6irdWg6DsvlDXIpmDnjpH_Mjx8c7t-ESX99zhwnhG8j1rGzq6whgCcxHGvpcIw8I-Jrr5l

In [30]:
def getReviewsAmazon(link):
    import requests
    from bs4 import BeautifulSoup
    from user_agent import generate_user_agent
    import re

    # Encabezados personalizados
    custom_headers = {
        'user-agent': generate_user_agent(),  # Generar un User-Agent aleatorio
        'accept-language': 'en-US,en;q=0.9',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
        'referer': 'https://www.amazon.com.mx/',
        'upgrade-insecure-requests': '1'
    }

    # Hacemos la solicitud GET a la página del producto
    response = requests.get(link, headers=custom_headers)
    
    # Comprobamos que la solicitud fue exitosa
    if response.status_code == 200:
        # Crear el objeto BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Buscar el enlace para ver todas las opiniones
        reviews_link_element = soup.find("a", {"data-hook": "see-all-reviews-link-foot"})
        
        # Verificar si se encontró el enlace
        if reviews_link_element and 'href' in reviews_link_element.attrs:
            # Construir la URL completa de la sección de opiniones
            reviews_link = "https://www.amazon.com.mx" + reviews_link_element['href']
            print("Enlace a todas las opiniones:", reviews_link)
            
            # Hacer una solicitud GET al enlace de opiniones
            reviews_response = requests.get(reviews_link, headers=custom_headers)
            
            # Comprobar que la solicitud fue exitosa
            if reviews_response.status_code == 200:
                reviews_soup = BeautifulSoup(reviews_response.text, 'html.parser')
                # Aquí procesamos la página de opiniones
                print("Página de opiniones accedida correctamente.")
                
                # Extraer el número de reseñas con opiniones
                review_info_element = reviews_soup.find("div", {"data-hook": "cr-filter-info-review-rating-count"})
                if review_info_element:
                    review_info_text = review_info_element.get_text(strip=True)
                    match = re.search(r"(\d{1,3}(?:,\d{3})*) con opiniones", review_info_text)
                    
                    if match:
                        reviews_with_opinions = int(match.group(1).replace(",", ""))
                        print(f"Número de reseñas con opiniones: {reviews_with_opinions}")

                        # Buscar todos los elementos de texto de reseñas
                        review_texts = reviews_soup.find_all("span", class_="a-size-base review-text review-text-content")
                        
                        # Almacenar los textos de reseña en una lista
                        reviews_content = []
                        for review in review_texts:
                            # Extraer el texto de la reseña
                            review_text = review.get_text(strip=True)
                            reviews_content.append(review_text)
                        
                        # Imprimir los textos de reseña encontrados
                        print("Textos de reseñas encontrados:")
                        for text in reviews_content:
                            print(text)
                        
                        return reviews_with_opinions, reviews_content
                    else:
                        print("No se encontró el número de reseñas con opiniones.")
                        return None
                else:
                    print("No se encontró el elemento con data-hook='cr-filter-info-review-rating-count'.")
                    return None
            else:
                print("No se pudo acceder a la página de opiniones. Status code:", reviews_response.status_code)
                return None
        else:
            print("No se encontró el enlace a todas las opiniones.")
            return None
    else:
        print("No se pudo acceder a la página del producto. Status code:", response.status_code)
        return None

getReviewsAmazon(product_links_list[0])



Enlace a todas las opiniones: https://www.amazon.com.mx/Hamilton-Beach-35034-Freidora/product-reviews/B00M39ML76/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews
Página de opiniones accedida correctamente.
Número de reseñas con opiniones: 3369
Textos de reseñas encontrados:
Frie muy parejitos los alimentos y funciona de maravilla , el tamaño es genial.
Recibí el producto con todas las piezas completas. Los materiales son resistentes y fáciles de usar. Además, tiene una gran capacidad de almacenamiento. La recomiendo para uso doméstico; podría servir para un negocio pequeño, pero no creo que los materiales soporten un uso intensivo.
Esto es más duradero y dura más que comprar algo más barato.
Muy buena freidora, de las mejores que hay por el precio, cocina de manera correcta los alimentos.
Fácil de usar
Funciona muy bien, excelente en todo.
Es la segunda que compro. La primera me duró aproximadamente 3 años de uso en un restaurante y volvería a comprar otras más.Tengo una pi

(3369,
 ['Frie muy parejitos los alimentos y funciona de maravilla , el tamaño es genial.',
  'Recibí el producto con todas las piezas completas. Los materiales son resistentes y fáciles de usar. Además, tiene una gran capacidad de almacenamiento. La recomiendo para uso doméstico; podría servir para un negocio pequeño, pero no creo que los materiales soporten un uso intensivo.',
  'Esto es más duradero y dura más que comprar algo más barato.',
  'Muy buena freidora, de las mejores que hay por el precio, cocina de manera correcta los alimentos.',
  'Fácil de usar',
  'Funciona muy bien, excelente en todo.',
  'Es la segunda que compro. La primera me duró aproximadamente 3 años de uso en un restaurante y volvería a comprar otras más.Tengo una pizzería y vendo papas a la francesa, boneless, nuggets y alitas. Duró más de 3 años hasta que se hizo carbón el conector magnético por el uso y el cable se dañó (cosa que tuvo solución y pude echarla a andar nuevamente soldando un cable a cada uno 